In [32]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split, RepeatedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [38]:
data = pd.read_csv("D:\FreeLance work\dataset_1.csv")

In [39]:
threshold = 0.2
missing_percentage = data.isnull().sum() / len(data)
filtered_data = data.loc[:, missing_percentage <= threshold]

In [ ]:
categorical_cols = filtered_data.select_dtypes(include=['object']).columns
numerical_cols = filtered_data.select_dtypes(exclude=['object']).columns

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    filtered_data[col] = le.fit_transform(filtered_data[col].astype(str))
    label_encoders[col] = le

imputer = IterativeImputer(max_iter=10, random_state=42)
imputed_data_numerical = pd.DataFrame(imputer.fit_transform(filtered_data[numerical_cols]), columns=numerical_cols)

imputed_data = pd.concat([imputed_data_numerical, filtered_data[categorical_cols]], axis=1)

print(imputed_data.head())

   subject_id  admission_type  insurance  race  los_hospital  gender   age  \
0  19912620.0             2.0        1.0   6.0          19.0     0.0  73.0   
1  16632367.0             2.0        3.0  23.0          13.0     1.0  61.0   
2  17812252.0             2.0        1.0  23.0           2.0     0.0  82.0   
3  16958639.0             5.0        3.0  23.0           8.0     1.0  70.0   
4  15275707.0             2.0        1.0   1.0          10.0     1.0  47.0   

   glucose_mean  heart_rate_mean    sbp_mean  ...  peptic_ulcer_disease  \
0    140.857143       100.291667  137.368421  ...                   0.0   
1    120.000000        90.851852  108.478261  ...                   0.0   
2    211.285714        82.173913  107.208333  ...                   0.0   
3    135.800000        77.592593  100.481482  ...                   0.0   
4    210.000000       123.375000  124.000000  ...                   0.0   

   mild_liver_disease  diabetes_without_cc  paraplegia  malignant_cancer  \
0   

In [ ]:
#Feature Selection using Recursive Feature Elimination
rfc = RandomForestClassifier(random_state=42)
# Using Recursive Feature Elimination (RFE)
rfe = RFE(estimator=rfc, n_features_to_select=10)  
X = imputed_data.drop('AKI', axis=1)
y = imputed_data['AKI']
# Fit RFE
rfe.fit(X, y)
selected_features = X.columns[rfe.support_]

In [ ]:
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

X = imputed_data.drop('AKI', axis=1)
vif_df = calculate_vif(X)
features_to_keep = vif_df[vif_df["VIF"] < 10]["Feature"].tolist()
X_reduced = X[features_to_keep]

y = imputed_data['AKI']
p_values = {}

for feature in X_reduced.columns:
    X_feature = X_reduced[feature]
    X_feature = sm.add_constant(X_feature) 
    try:
        logit_model = sm.Logit(y, X_feature).fit(disp=0)  
        p_value = logit_model.pvalues[1] 
        p_values[feature] = p_value
    except np.linalg.LinAlgError:
        print(f"Skipping {feature} due to singular matrix issue.")

# Display p-values
for feature, p_value in p_values.items():
    print(f"{feature}: {p_value}")

Skipping paraplegia due to singular matrix issue.
subject_id: 0.19525455924866542
admission_type: 0.16203199568525684
insurance: 0.3645877173525739
race: 0.1697438367341948
los_hospital: 1.8809954118754236e-11
gender: 0.7675223049671472
age: 0.07122517213744214
glucose_mean: 1.7922219774656192e-05
heart_rate_mean: 7.670048920393652e-06
sbp_mean: 0.0012882081166762168
resp_rate_mean: 2.6061681912358655e-05
temperature_mean: 0.8658507748639903
spo2_mean: 0.29586994410556533
platelets_max: 0.128861653291366
wbc_max: 0.00019545172251663566
aniongap_min: 0.0037031857726631673
bicarbonate_max: 0.066653522247466
bun_max: 5.736309050374102e-10
calcium_max: 0.8741627556393585
chloride_max: 0.43587865357496147
creatinine_max: 5.74495683462442e-09
glucose_max: 5.06857577950663e-05
sodium_max: 0.3006578831178357
potassium_max: 3.6855261953568034e-05
ptt_max: 0.00784327627260694
alt_max: 0.2411545567605442
alp_max: 0.9726235392626343
ast_max: 0.12018715114799997
bilirubin_total_max: 0.5935736619375

C:\Users\shahe\AppData\Local\Temp\ipykernel_10680\3895237841.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = logit_model.pvalues[1]  # Extract p-value of the feature
C:\Users\shahe\AppData\Local\Temp\ipykernel_10680\3895237841.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = logit_model.pvalues[1]  # Extract p-value of the feature
C:\Users\shahe\AppData\Local\Temp\ipykernel_10680\3895237841.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a v

In [ ]:
#Splitting data into training and testing sets
X_selected = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

In [ ]:
# Training Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
# Making predictions
y_pred = rf_model.predict(X_test)
y_pred_prob = rf_model.predict_proba(X_test)[:, 1]

# Evaluating the model
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC Score: {auc}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Step 5: Visualize ROC Curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (AUC = {:.2f})'.format(auc))
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()